# MazeruHAR 动态训练引擎

欢迎使用 MazeruHAR 的配置驱动训练引擎。此 Notebook 旨在提供一个灵活、可重现且易于使用的训练流程。

**核心理念:** **代码只写一次，实验配置万千。**

您只需要执行以下两个简单步骤即可开始训练：

1.  **配置实验**: 修改位于 `config/` 目录下的 `.yaml` 配置文件。您可以复制 `config/default_configs/shl_config.yaml` 并根据您的需求进行调整，比如更换数据集、模型架构或超参数。
2.  **运行 Notebook**: 在下面的 **“实验配置”**单元格中设置好配置文件的路径，然后从头到尾运行此 Notebook 即可。

---

## 步骤 1: 环境设置与库导入

此单元格负责导入所有必需的库并设置初始环境。它整合了项目所需的所有依赖项。

In [ ]:
# 标准库导入
import os
import sys
import time
import yaml
import random
import logging
import json
from pathlib import Path
from typing import Dict, Any, Optional, List, Tuple

# 第三方库导入
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR, ReduceLROnPlateau
from sklearn.metrics import f1_score, confusion_matrix, classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# --- 项目内部模块导入 ---
print("所有模块已准备就绪。")

## 步骤 2: 实验配置

**这是您唯一需要修改的单元格。**

请将 `CONFIG_PATH`变量设置为您想要使用的配置文件的路径。所有实验参数都将从此文件加载。

In [ ]:
# ================== 核心配置 ==================
# 🔥 只需修改此处的配置文件路径即可开始新的实验
CONFIG_PATH = 'config/default_configs/shl_config.yaml'  # 默认使用SHL配置文件
# ==============================================

# 检查配置文件是否存在
if not os.path.exists(CONFIG_PATH):
    print(f"❌ 错误: 配置文件 '{CONFIG_PATH}' 未找到!")
    print("请确保路径正确，或创建一个新的配置文件。")
else:
    print(f"✓ 将使用配置文件: '{CONFIG_PATH}'")

## 步骤 3: 核心训练器类

下面的 `ConfigurableTrainer` 类是整个训练流程的核心。它封装了从配置加载、环境设置、数据处理、模型创建、训练、评估到结果可视化的所有逻辑。您无需修改此类中的任何代码。

In [ ]:
class ConfigurableTrainer:
    def __init__(self, config_path: str):
        """初始化训练器"""
        self.config_path = config_path
        self.config = self._load_config(config_path)
        self.params = self._extract_params()
        self.history = {
            'train_loss': [], 'train_accuracy': [],
            'val_loss': [], 'val_accuracy': [], 'val_f1': []
        }
        self.model = None
        self.optimizer = None
        self.scheduler = None
        self.criterion = None
        self.device = None
        self.logger = None

    def _load_config(self, config_path: str):
        """加载配置文件"""
        import yaml
        with open(config_path, 'r', encoding='utf-8') as f:
            return yaml.safe_load(f)

    def _extract_params(self):
        """提取参数"""
        params = {}
        # 数据集参数
        dataset_config = self.config.get('dataset', {})
        params.update({
            'name': dataset_config.get('name', 'SHL'),
            'path': dataset_config.get('path', './datasets/datasetStandardized/SHL_Multimodal/'),
            'window_size': dataset_config.get('preprocessing', {}).get('window_size', 128),
            'step_size': dataset_config.get('preprocessing', {}).get('step_size', 64),
            'sample_rate': dataset_config.get('preprocessing', {}).get('sample_rate', 100),
            'modalities': dataset_config.get('modalities', {}),
            'activity_labels': list(dataset_config.get('activity_labels', {}).values())
        })
        
        # 训练参数
        training_config = self.config.get('training', {})
        params.update({
            'epochs': training_config.get('epochs', 100),
            'batch_size': training_config.get('batch_size', 64),  # 增加批大小
            'learning_rate': training_config.get('learning_rate', 0.0005),  # 降低学习率
            'weight_decay': training_config.get('weight_decay', 0.01),  # 增加正则化
            'optimizer_name': training_config.get('optimizer', 'AdamW'),
            'scheduler_name': training_config.get('scheduler', 'step'),
            'label_smoothing': training_config.get('label_smoothing', 0.05),  # 降低标签平滑
            'gradient_clip_norm': training_config.get('gradient_clip_norm', 1.0),
            'early_stopping_patience': training_config.get('early_stopping', {}).get('patience', 25)
        })
        
        # 环境参数
        env_config = self.config.get('environment', {})
        params.update({
            'seed': env_config.get('seed', 42),
            'device': env_config.get('device', 'auto'),
            'num_workers': env_config.get('num_workers', 4)
        })
        
        # 输出配置
        params.update({
            'output_dir': f"./results/{params['name']}_fixed",
            'save_checkpoints': True,
            'verbose': True
        })
        
        return params

    def setup_environment(self):
        """设置环境和日志记录"""
        import random
        
        # 设置随机种子
        seed = self.params.get('seed', 42)
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
        
        # 设置设备
        device_pref = self.params.get('device', 'auto')
        if device_pref == 'auto':
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = torch.device(device_pref)
        
        # 创建输出目录
        self.output_dir = Path(self.params.get('output_dir', './results/shl_fixed'))
        self.output_dir.mkdir(parents=True, exist_ok=True)

        # 设置日志
        logging.basicConfig(
            level=logging.INFO, 
            format='%(asctime)s [%(levelname)s] - %(message)s',
            handlers=[
                logging.FileHandler(self.output_dir / 'training.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger()
        self.logger.info(f"环境设置完成。设备: {self.device}, 输出目录: {self.output_dir}")

    def load_data(self):
        """修复数据加载逻辑"""
        self.logger.info(f"加载SHL数据集: {self.params['name']}")
        
        # 引入数据层模块
        from data_layer import SHLDataParser, UniversalHARDataset
        
        # 配置数据解析器
        parser_config = {
            'name': 'SHL',
            'data_path': self.params.get('path', './datasets/datasetStandardized/SHL_Multimodal/'),
            'window_size': self.params.get('window_size', 128),
            'step_size': self.params.get('step_size', 64),
            'sample_rate': self.params.get('sample_rate', 100),
            'normalize_per_sample': True,  # 启用样本级标准化
            'modalities': {
                'imu': {'enabled': True, 'channels': 6},
                'pressure': {'enabled': True, 'channels': 1}
            }
        }
        
        # 创建数据解析器和数据集
        parser = SHLDataParser(parser_config)
        train_dataset = UniversalHARDataset(parser, split='train')
        dev_dataset = UniversalHARDataset(parser, split='val')
        test_dataset = UniversalHARDataset(parser, split='test')
        
        self.logger.info(f"数据集大小 - 训练: {len(train_dataset)}, 验证: {len(dev_dataset)}, 测试: {len(test_dataset)}")
        
        # 创建数据加载器
        train_loader = DataLoader(
            train_dataset, 
            batch_size=self.params['batch_size'],
            shuffle=True,
            num_workers=self.params['num_workers'],
            pin_memory=True,
            drop_last=True  # 确保批大小一致
        )
        
        dev_loader = DataLoader(
            dev_dataset, 
            batch_size=self.params['batch_size'],
            shuffle=False,
            num_workers=self.params['num_workers'],
            pin_memory=True
        )
        
        test_loader = DataLoader(
            test_dataset, 
            batch_size=self.params['batch_size'],
            shuffle=False,
            num_workers=self.params['num_workers'],
            pin_memory=True
        )
        
        # 收集训练标签用于计算类别权重
        self.logger.info("收集训练标签...")
        train_labels = []
        for _, labels in train_loader:
            train_labels.extend(labels.numpy())
        train_labels = np.array(train_labels)
        
        # 打印类别分布
        unique_labels, counts = np.unique(train_labels, return_counts=True)
        self.logger.info(f"类别分布: {dict(zip(unique_labels, counts))}")
        
        return train_loader, dev_loader, test_loader, train_labels

    def build_model(self):
        """构建动态模型"""
        self.logger.info("构建动态HAR模型...")
        
        from model_layer.dynamic_har_model import DynamicHarModel
        
        # 修复模型配置
        model_config = self.config.copy()
        # 确保architecture配置正确
        if 'architecture' in model_config:
            arch_config = model_config['architecture']
            # 修正专家配置中的类型名称
            if 'experts' in arch_config:
                for expert_name, expert_config in arch_config['experts'].items():
                    if 'type' in expert_config:
                        # 确保类型名称小写且正确
                        expert_type = expert_config['type'].lower()
                        if expert_type in ['transformerexpert', 'transformer_expert']:
                            expert_config['type'] = 'transformer'
                        elif expert_type in ['rnnexpert', 'rnn_expert']:
                            expert_config['type'] = 'rnn'
                        elif expert_type in ['cnnexpert', 'cnn_expert']:
                            expert_config['type'] = 'cnn'
        
        self.model = DynamicHarModel(model_config)
        self.model.to(self.device)
        
        # 打印模型信息
        total_params = sum(p.numel() for p in self.model.parameters())
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        self.logger.info(f"模型参数总数: {total_params:,}")
        self.logger.info(f"可训练参数: {trainable_params:,}")

    def setup_training_components(self, train_labels: np.ndarray):
        """设置训练组件（优化器、损失函数等）"""
        self.logger.info("设置训练组件...")
        
        # 计算类别权重以处理数据不平衡
        classes = np.unique(train_labels)
        class_weights = compute_class_weight(
            'balanced', 
            classes=classes, 
            y=train_labels
        )
        class_weights_tensor = torch.FloatTensor(class_weights).to(self.device)
        self.logger.info(f"类别权重: {dict(zip(classes, class_weights))}")
        
        # 损失函数
        self.criterion = nn.CrossEntropyLoss(
            weight=class_weights_tensor,
            label_smoothing=self.params['label_smoothing']
        )
        
        # 优化器
        optimizer_name = self.params['optimizer_name'].lower()
        if optimizer_name == 'adamw':
            self.optimizer = optim.AdamW(
                self.model.parameters(),
                lr=self.params['learning_rate'],
                weight_decay=self.params['weight_decay'],
                betas=(0.9, 0.999),
                eps=1e-8
            )
        elif optimizer_name == 'adam':
            self.optimizer = optim.Adam(
                self.model.parameters(),
                lr=self.params['learning_rate'],
                weight_decay=self.params['weight_decay']
            )
        else:
            raise ValueError(f"不支持的优化器: {optimizer_name}")
        
        # 学习率调度器
        scheduler_name = self.params['scheduler_name']
        if scheduler_name == 'step':
            self.scheduler = optim.lr_scheduler.StepLR(
                self.optimizer, 
                step_size=50, 
                gamma=0.5
            )
        elif scheduler_name == 'cosine':
            self.scheduler = optim.lr_scheduler.CosineAnnealingLR(
                self.optimizer, 
                T_max=self.params['epochs']
            )
        elif scheduler_name == 'plateau':
            self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, 
                mode='min', 
                patience=10,
                factor=0.5
            )
        else:
            self.scheduler = None
        
        self.logger.info(f"训练组件设置完成: 优化器={optimizer_name}, 调度器={scheduler_name}")

    def train(self, train_loader, dev_loader):
        """修复训练循环"""
        self.logger.info("开始训练...")
        
        best_val_f1 = 0.0
        patience_counter = 0
        patience = self.params['early_stopping_patience']
        
        for epoch in range(self.params['epochs']):
            # 训练阶段
            self.model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0
            
            for batch_idx, (data_dict, targets) in enumerate(train_loader):
                # 将数据移到设备
                data_dict = {k: v.to(self.device) for k, v in data_dict.items()}
                targets = targets.to(self.device)
                
                # 前向传播
                self.optimizer.zero_grad()
                
                try:
                    outputs = self.model(data_dict)
                    loss = self.criterion(outputs, targets)
                except Exception as e:
                    self.logger.error(f"前向传播错误: {e}")
                    # 打印调试信息
                    for k, v in data_dict.items():
                        self.logger.error(f"输入 {k} 形状: {v.shape}")
                    raise e
                
                # 反向传播
                loss.backward()
                
                # 梯度裁剪
                if self.params['gradient_clip_norm'] > 0:
                    torch.nn.utils.clip_grad_norm_(
                        self.model.parameters(),
                        self.params['gradient_clip_norm']
                    )
                
                self.optimizer.step()
                
                # 统计
                train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                train_total += targets.size(0)
                train_correct += (predicted == targets).sum().item()
                
                # 每100个batch打印一次进度
                if batch_idx % 100 == 0:
                    self.logger.info(f"Epoch {epoch+1}/{self.params['epochs']}, "
                                      f"Batch {batch_idx}/{len(train_loader)}, "
                                      f"Loss: {loss.item():.4f}")
            
            # 更新学习率
            if self.scheduler and not isinstance(self.scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                self.scheduler.step()
            
            # 验证阶段
            val_loss, val_f1, val_acc = self.evaluate(dev_loader, is_test=False)
            train_acc = train_correct / train_total
            train_loss_avg = train_loss / len(train_loader)
            
            # 如果使用ReduceLROnPlateau，在这里更新
            if isinstance(self.scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                self.scheduler.step(val_loss)
            
            # 记录历史
            self.history['train_loss'].append(train_loss_avg)
            self.history['train_accuracy'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_accuracy'].append(val_acc)
            self.history['val_f1'].append(val_f1)
            
            # 打印训练信息
            current_lr = self.optimizer.param_groups[0]['lr']
            self.logger.info(
                f"Epoch {epoch+1}/{self.params['epochs']} | "
                f"Train Loss: {train_loss_avg:.4f}, Train Acc: {train_acc:.4f} | "
                f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f} | "
                f"LR: {current_lr:.6f}"
            )
            
            # 保存最佳模型
            if val_f1 > best_val_f1:
                best_val_f1 = val_f1
                patience_counter = 0
                if self.params.get('save_checkpoints', True):
                    torch.save(self.model.state_dict(), self.output_dir / 'best_model.pth')
                    self.logger.info(f"新最佳模型已保存，验证F1分数: {best_val_f1:.4f}")
            else:
                patience_counter += 1
            
            # 早停检查
            if patience_counter >= patience:
                self.logger.info(f"早停触发! 最佳验证F1: {best_val_f1:.4f}")
                break
        
        self.logger.info(f"训练完成! 最佳验证F1: {best_val_f1:.4f}")

    def evaluate(self, data_loader, is_test=False):
        """评估方法 - 支持多模态数据"""
        self.model.eval()
        total_loss = 0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for data_dict, targets in data_loader:
                data_dict = {k: v.to(self.device) for k, v in data_dict.items()}
                targets = targets.to(self.device)
                
                try:
                    outputs = self.model(data_dict)
                    loss = self.criterion(outputs, targets)
                except Exception as e:
                    self.logger.error(f"评估时前向传播错误: {e}")
                    continue
                
                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                
                all_preds.extend(predicted.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
        
        avg_loss = total_loss / len(data_loader)
        accuracy = accuracy_score(all_targets, all_preds)
        f1 = f1_score(all_targets, all_preds, average='weighted')
        
        if is_test:
            # 打印详细的测试结果
            from sklearn.metrics import classification_report, confusion_matrix
            self.logger.info("测试集详细结果:")
            self.logger.info(f"准确率: {accuracy:.4f}")
            self.logger.info(f"F1分数: {f1:.4f}")
            
            # 分类报告
            target_names = self.params.get('activity_labels', [f'Class_{i}' for i in range(8)])
            report = classification_report(all_targets, all_preds, target_names=target_names)
            self.logger.info(f"分类报告:\n{report}")
            
            # 混淆矩阵
            cm = confusion_matrix(all_targets, all_preds)
            self.logger.info(f"混淆矩阵:\n{cm}")
        
        return avg_loss, f1, accuracy

    def plot_learning_curves(self):
        """绘制学习曲线"""
        import matplotlib.pyplot as plt
        
        plt.figure(figsize=(15, 5))
        
        # 损失曲线
        plt.subplot(1, 3, 1)
        plt.plot(self.history['train_loss'], label='Train Loss')
        plt.plot(self.history['val_loss'], label='Validation Loss')
        plt.title('Loss vs. Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        
        # 准确率曲线
        plt.subplot(1, 3, 2)
        plt.plot(self.history['train_accuracy'], label='Train Accuracy')
        plt.plot(self.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Accuracy vs. Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)
        
        # F1分数曲线
        plt.subplot(1, 3, 3)
        plt.plot(self.history['val_f1'], label='Validation F1', color='orange')
        plt.title('F1 Score vs. Epochs')
        plt.xlabel('Epoch')
        plt.ylabel('F1 Score')
        plt.legend()
        plt.grid(True)
        
        plt.tight_layout()
        save_path = self.output_dir / 'learning_curves.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        self.logger.info(f"学习曲线已保存至 {save_path}")
        plt.show()

    def plot_confusion_matrix(self, y_true, y_pred, labels):
        """绘制混淆矩阵"""
        import matplotlib.pyplot as plt
        import seaborn as sns
        from sklearn.metrics import confusion_matrix
        
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=labels, yticklabels=labels)
        plt.title('Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        save_path = self.output_dir / 'confusion_matrix.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        self.logger.info(f"混淆矩阵已保存至 {save_path}")
        plt.show()

    def validate_shl_config(self):
        """验证SHL配置文件的完整性"""
        required_sections = ['dataset', 'architecture', 'training']
        
        for section in required_sections:
            if section not in self.config:
                raise ValueError(f"配置文件缺少必需的 '{section}' 部分")
        
        # 验证数据集配置
        dataset_config = self.config['dataset']
        if dataset_config['name'] != 'SHL':
            raise ValueError("配置文件不是为SHL数据集设计的")
        
        # 验证模态配置
        modalities = dataset_config.get('modalities', {})
        required_modalities = ['imu', 'pressure']
        for modality in required_modalities:
            if modality not in modalities or not modalities[modality].get('enabled', False):
                raise ValueError(f"SHL配置必须启用 '{modality}' 模态")
        
        # 验证专家配置
        experts = self.config['architecture']['experts']
        expected_experts = ['imu_expert', 'pressure_expert']
        for expert in expected_experts:
            if expert not in experts:
                raise ValueError(f"配置文件缺少必需的专家模型: {expert}")
        
        self.logger.info("✓ SHL配置文件验证通过")

    def run(self):
        """完整的训练流程 - SHL多模态版本"""
        try:
            self.setup_environment()
            
            # 强制验证配置
            self.validate_shl_config()
            
            self.logger.info("开始SHL多模态训练实验")
            self.logger.info(f"使用配置: {self.config_path}")
            
            # 加载数据
            train_loader, dev_loader, test_loader, train_labels = self.load_data()
            
            # 构建模型
            self.build_model()
            
            # 设置训练组件
            self.setup_training_components(train_labels)
            
            # 训练
            self.train(train_loader, dev_loader)
            
            # 绘制学习曲线
            self.plot_learning_curves()
            
            # 最终测试
            self.logger.info("进行最终测试...")
            
            # 加载最佳模型
            best_model_path = self.output_dir / 'best_model.pth'
            if best_model_path.exists():
                self.model.load_state_dict(torch.load(best_model_path))
                self.logger.info("已加载最佳模型进行测试")
            
            test_loss, test_f1, test_acc = self.evaluate(test_loader, is_test=True)
            self.logger.info(f"最终测试结果 - Loss: {test_loss:.4f}, Acc: {test_acc:.4f}, F1: {test_f1:.4f}")
            
            self.logger.info("SHL多模态训练实验完成!")
            
        except Exception as e:
            self.logger.error(f"训练过程中发生错误: {str(e)}")
            import traceback
            self.logger.error(f"错误堆栈:\n{traceback.format_exc()}")
            raise e

## 步骤 4: 执行训练流程

最后，我们实例化 `ConfigurableTrainer` 类并调用其 `run` 方法来启动整个训练和评估流程。所有操作都将由之前加载的配置驱动。

In [ ]:
# 主执行代码
if __name__ == '__main__':
    try:
        # 使用修复后的SHL配置文件
        config_file = 'config/default_configs/shl_config.yaml'
        
        if not os.path.exists(config_file):
            print(f"❌ SHL配置文件 {config_file} 不存在!")
            print("请先创建SHL数据集的配置文件")
        else:
            print(f"✓ 使用SHL配置文件: {config_file}")
            trainer = ConfigurableTrainer(config_path=config_file)
            trainer.run()
        
    except KeyboardInterrupt:
        print("\n用户中断训练")
    except Exception as e:
        print(f"\nSHL训练流程发生错误: {str(e)}")
        import traceback
        traceback.print_exc()